In [1]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# 1) Imports & Device Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2) Custom Dataset Definition
class RegionDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['filename'])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(row['Region_ID'] - 1, dtype=torch.long)
        return image, label, row['filename']

# 3) Data Transform Pipelines
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

# 4) Loading DataFrames
train_csv = '/kaggle/input/smai-project/Phase_2_data/labels_train.csv'
val_csv   = '/kaggle/input/smai-project/Phase_2_data/labels_val.csv'
train_img_dir = '/kaggle/input/smai-project/Phase_2_data/images_train'
val_img_dir   = '/kaggle/input/smai-project/Phase_2_data/images_val'

train_df = pd.read_csv(train_csv)
val_df   = pd.read_csv(val_csv)

# 5) Dataset & DataLoader Construction
batch_size = 32
train_dataset = RegionDataset(train_df, train_img_dir, train_transform)
val_dataset   = RegionDataset(val_df,   val_img_dir,   val_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=4)

# 6) Model Builder Function
def build_model(name='efficientnet_b0', num_classes=15, pretrained=True):
    if name == 'efficientnet_b0':
        model = models.efficientnet_b0(pretrained=pretrained)
        in_features = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(in_features, num_classes)
    elif name == 'resnet50':
        model = models.resnet50(pretrained=pretrained)
        in_features = model.fc.in_features
        model.fc = nn.Linear(in_features, num_classes)
    elif name == 'convnext_tiny':
        model = models.convnext_tiny(pretrained=pretrained)
        in_features = model.classifier[2].in_features
        model.classifier[2] = nn.Linear(in_features, num_classes)
    elif name == 'mobilenet_v3_large':
        model = models.mobilenet_v3_large(pretrained=pretrained)
        in_features = model.classifier[3].in_features
        model.classifier[3] = nn.Linear(in_features, num_classes)
    else:
        raise ValueError(f"Unsupported model: {name}")
    return model.to(device)

# 7) Training & Validation Loop
def train_validate(model, train_loader, val_loader, epochs=10, model_name='model'):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-6)
    best_acc = 0.0
    best_model_path = f"best_{model_name}.pth"

    for epoch in range(epochs):
        # Training
        model.train()
        train_loss, all_preds, all_labels = 0.0, [], []
        for images, labels, _ in tqdm(train_loader, desc=f"Train Epoch {epoch+1}/{epochs}"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1).cpu().tolist()
            all_preds += preds
            all_labels += labels.cpu().tolist()

        scheduler.step()
        train_loss /= len(train_loader.dataset)
        train_acc = accuracy_score(all_labels, all_preds)

        # Validation
        model.eval()
        val_loss, all_preds, all_labels, filenames = 0.0, [], [], []
        with torch.no_grad():
            for images, labels, fn in tqdm(val_loader, desc="Validate"):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                preds = outputs.argmax(dim=1).cpu().tolist()
                all_preds += preds
                all_labels += labels.cpu().tolist()
                filenames += fn

        val_loss /= len(val_loader.dataset)
        val_acc = accuracy_score(all_labels, all_preds)

        # Checkpointing best model
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), best_model_path)
            pd.DataFrame({'filename': filenames,
                          'pred_region': [p+1 for p in all_preds]}) \
              .to_csv(f"best_{model_name}_predictions.csv", index=False)

        print(f"Epoch {epoch+1}/{epochs} | "
              f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

    print(f"Best Validation Accuracy: {best_acc:.4f}")
    print(f"Best model saved to: {best_model_path}")
    return best_acc

# 8) Main Execution
if __name__ == "__main__":
    # Choose from: 'efficientnet_b0', 'resnet50', 'convnext_tiny', 'mobilenet_v3_large'
    chosen_model = 'convnext_tiny'
    model = build_model(name=chosen_model, num_classes=15, pretrained=True)
    _ = train_validate(model, train_loader, val_loader,
                       epochs=100, model_name=chosen_model)



/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Tiny_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/convnext_tiny-983f1562.pth" to /root/.cache/torch/hub/checkpoints/convnext_tiny-983f1562.pth
100%|██████████| 109M/109M [00:00<00:00, 196MB/s] 
Validate: 100%|██████████| 12/12 [00:02<00:00,  5.45it/s]


Epoch 1/100 | Train Loss: 1.5922, Acc: 0.4853 | Val Loss: 0.7233, Acc: 0.7669


Validate: 100%|██████████| 12/12 [00:02<00:00,  5.66it/s]


Epoch 2/100 | Train Loss: 0.8922, Acc: 0.7119 | Val Loss: 0.6617, Acc: 0.7724


Validate: 100%|██████████| 12/12 [00:02<00:00,  5.62it/s]


...

Epoch 98/100 | Train Loss: 0.0487, Acc: 0.9833 | Val Loss: 0.1460, Acc: 0.9783


Validate: 100%|██████████| 12/12 [00:02<00:00,  5.67it/s]


Epoch 99/100 | Train Loss: 0.0456, Acc: 0.9852 | Val Loss: 0.1452, Acc: 0.9783


Validate: 100%|██████████| 12/12 [00:02<00:00,  5.67it/s]

Epoch 100/100 | Train Loss: 0.0533, Acc: 0.9830 | Val Loss: 0.1440, Acc: 0.9783
Best Validation Accuracy: 0.9837
Best model saved to: best_convnext_tiny.pth
